In [ ]:
import folium

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from folium.plugins import MarkerCluster


In [ ]:
m = folium.Map(location=[51.5, -0.1], zoom_start=15)

In [ ]:
m

In [ ]:
accident_data = pd.read_csv("dataWith5000RNRecordsWithIDAndHeaders")
accident_data = accident_data.sample(n=5000, random_state=42)
accident_data.dropna(subset=["Latitude", "Longitude"], inplace=True)

In [ ]:
accident_data.head(5)

In [ ]:
accident_data.shape

In [ ]:
def get_geojson_grid(upper_right, lower_left, n=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [ ]:
 def generate_heat_gridmap(accident_data, htmlfilename):
    m = folium.Map(zoom_start = 5, location=[55, 0])

    # Generate GeoJson grid
    top_right = [58, 2]
    top_left = [49, -8]

    grid = get_geojson_grid(top_right, top_left, n=6)

    # Calculate exposures in grid
    #popups = []
    regional_counts = []

    for box in grid:
        upper_right = box["properties"]["upper_right"]
        lower_left = box["properties"]["lower_left"]

        mask = (
            (accident_data.Latitude != upper_right[1]) & (accident_data.Latitude != lower_left[1]) &
            (accident_data.Longitude != upper_right[0]) & (accident_data.Longitude != lower_left[0])
               )

        region_incidents = len(accident_data[mask])
        regional_counts.append(region_incidents)

        #total_vehicles = accident_data[mask].Number_of_Vehicles.sum()
        #total_casualties = accident_data[mask].Number_of_Casualties.sum()
        #content = "total vehicles {:,.0f}, total casualties {:,.0f}".format(total_vehicles, total_casualties)
        #popup = folium.Popup(content)
        #popups.append(popup)

    worst_region = max(regional_counts)

    # Add GeoJson to map
    for i, box in enumerate(grid):
        geo_json = json.dumps(box)

        color = plt.cm.Reds(regional_counts[i] / worst_region)
        color = mpl.colors.to_hex(color)

        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature, color=color: {
                                                                            'fillColor': color,
                                                                            'color':"black",
                                                                            'weight': 2,
                                                                            'dashArray': '5, 5',
                                                                            'fillOpacity': 0.55,
                                                                        })

        #gj.add_child(popups[i])
        m.add_child(gj)

    # Marker clusters
    locations = list(zip(accident_data.Latitude, accident_data.Longitude))
    icons = [folium.Icon(icon="car", prefix="fa") for _ in range(len(locations))]

    # Create popups
    popup_content = []
    for incident in accident_data.itertuples():
        #number_of_vehicles = "Number of vehicles: {} <br/>".format(incident.Number_of_Vehicles)
        #number_of_casualties = "Number of casualties: {} <br/>".format(incident.Number_of_Casualties)
        weather_conditions = "Weather Condition: {} <br/>".format(incident.Weather_Conditions)
        triangle_count = "Triangle Count: {} ".format(incident.Triangle_Count)
        #content = number_of_vehicles + number_of_casualties + weather_conditions + triangle_count
        content = weather_conditions + triangle_count
        popup_content.append(content)

    popups = [folium.Popup(content) for content in popup_content]

    cluster = MarkerCluster(locations=locations, icons=icons, popups=popups)
    m.add_child(cluster)

    m.save(htmlfilename)

In [ ]:
Weather_100_data = pd.read_csv("Weather_100_TriangleCount.csv")
Weather_100_data.head(5)

In [ ]:
Weather_500_data = pd.read_csv("Weather_500_TriangleCount.csv")
Weather_500_data.head(5)

In [ ]:
Weather_1000_data = pd.read_csv("Weather_1000_TriangleCount.csv")
Weather_1000_data.head(5)

In [ ]:
required_columns = list(accident_data.columns) + ['Triangle_Count']

In [ ]:
accident_weather_100_data = pd.merge(accident_data, Weather_100_data, left_on='Accident_ID', right_on='Node', how='inner')

In [ ]:
accident_weather_100 = accident_weather_100_data[required_columns].sort_values('Triangle_Count', ascending=False)
accident_weather_100.head(5)

In [ ]:
accident_weather_100_top5 = accident_weather_100.head(5)
accident_weather_100.head(5)

In [ ]:
generate_heat_gridmap(accident_weather_100_top5, "car_accidents_weather_100_v1.html")

In [ ]:
accident_weather_500_data = pd.merge(accident_data, Weather_500_data, left_on='Accident_ID', right_on='Node', how='inner')

In [ ]:
accident_weather_500 = accident_weather_500_data[required_columns].sort_values('Triangle_Count', ascending=False)

In [ ]:
accident_weather_500_top5 = accident_weather_500.head(5)
accident_weather_500.head(5)

In [ ]:
generate_heat_gridmap(accident_weather_500_top5, "car_accidents_weather_500_v1.html")

In [ ]:
accident_weather_1000_data = pd.merge(accident_data, Weather_1000_data, left_on='Accident_ID', right_on='Node', how='inner')

In [ ]:
accident_weather_1000 = accident_weather_1000_data[required_columns].sort_values('Triangle_Count', ascending=False)

In [ ]:
accident_weather_1000_top5 = accident_weather_1000.head(5)
accident_weather_1000.head(5)

In [ ]:
generate_heat_gridmap(accident_weather_1000_top5, "car_accidents_weather_1000_v1.html")